In [1]:
# import os
# from dotenv import load_dotenv

# load_dotenv()

# BRONZE_PATH = os.environ.get("BRONZE_PATH")
# SILVER_PATH = os.environ.get("SILVER_PATH")
# MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY")
# MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY")

# bronze_path = f"s3a://{BRONZE_PATH}/vnstock3/stock_quote_history_daily/"
# silver_path = f"s3a://{SILVER_PATH}/vnstock3/daily_stock_prices/"

# endpoint_url = "http://localhost:9000"

# bronze_path

In [2]:
bronze_path = f"s3a://dev/data/bronze/vnstock3/stock_quote_history_daily/"
silver_path = f"s3a://data/silver/vnstock3/daily_stock_prices/"

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Setup Spark sesssion
spark = (SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("DailyStockPriceProcessor")
    .getOrCreate()
)

spark

In [5]:
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.app.startTime', '1730802195058'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.2-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.2

In [6]:
# Example: Creating a DataFrame
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Id"])
df

df.show()

+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
|Cathy|  3|
+-----+---+



In [7]:
sc = spark.sparkContext

rdd = sc.parallelize([1, 4, 2, 2, 3, 4])
# 1) sử dụng hàm map() biến đổi rdd phía trên thành [2, 4, 6, 8]
# Ví dụ
rdd.map(lambda x: x * 2).collect()

[2, 8, 4, 4, 6, 8]

In [8]:
# spark.sparkContext.getConf().getAll()

In [10]:
# Define the S3 bucket path
bucket_name = "dev"  # Replace with your actual bucket name
bucket_path = f"s3a://{bucket_name}/"

# Check if the bucket path is valid
if not bucket_name:
    raise ValueError("Bucket name is null or empty.")

# Read the contents of the S3 bucket
try:
    df = spark.read.format("text").load(bucket_path)
    # Show the contents
    df.show(truncate=False)
except Exception as e:
    print(f"Error reading from bucket: {e}")

Error reading from bucket: An error occurred while calling o153.load.
: java.lang.AssertionError: assertion failed: Conflicting directory structures detected. Suspicious paths
	s3a://dev/data/bronze/vnstock3/stock_quote_history_daily
	s3a://dev/data/bronze/vnstock3/symbol_exchange
	s3a://dev/data/bronze/vnstock3/symbol_industry
	s3a://dev/metadata

If provided paths are partition directories, please set "basePath" in the options of the data source to specify the root directory of the table. If there are multiple root directories, please load them separately and then union them.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.sql.execution.datasources.PartitioningUtils$.parsePartitions(PartitioningUtils.scala:177)
	at org.apache.spark.sql.execution.datasources.PartitioningUtils$.parsePartitions(PartitioningUtils.scala:109)
	at org.apache.spark.sql.execution.datasources.PartitioningAwareFileIndex.inferPartitioning(PartitioningAwareFileIndex.scala:191)
	at org.apache.spark

In [11]:
bronze_path = 's3a://dev/data/bronze/vnstock3/stock_quote_history_daily/'

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, IntegerType

schema = StructType([
    StructField("time", TimestampType(), True),
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True),
    StructField("volume", IntegerType(), True),
    StructField("symbol", StringType(), True),
    StructField("loaded_timestamp", TimestampType(), True)
])

# Read the raw data from the bronze path in MinIO
raw_df = spark.read.schema(schema).json(bronze_path)

In [13]:
raw_df.show()

+--------------------+----+----+----+-----+------+------+--------------------+----------+
|                time|open|high| low|close|volume|symbol|    loaded_timestamp|updated_at|
+--------------------+----+----+----+-----+------+------+--------------------+----------+
|+32560-08-20 00:0...| 3.5| 3.5| 3.5|  3.5|  1500|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32568-11-06 00:0...|3.56|3.56|3.56| 3.56|  3000|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32574-04-29 00:0...|3.63|3.63|3.63| 3.63|  2100|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32579-10-20 00:0...| 3.7| 3.7| 3.7|  3.7|  2300|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32588-01-06 00:0...|3.76|3.76|3.76| 3.76|  1500|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32593-06-28 00:0...|3.83|3.83|3.83| 3.83|  7600|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32598-12-19 00:0...|3.89|3.89|3.89| 3.89|  1400|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32607-03-08 00:0...|3.96|3.96|3.96| 3.96|  1000|   HAP|+56805-05-30 08:4...|2024-11-01|
|+32612-08

In [14]:
raw_df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: integer (nullable = true)
 |-- symbol: string (nullable = true)
 |-- loaded_timestamp: timestamp (nullable = true)
 |-- updated_at: date (nullable = true)



In [15]:
raw_df.count()

1818063